# Multi-hop RAG Preprocessing for Complex PDFs

This notebook walks you through building a multi-hop RAG engine using Snowflake. Multi-hop RAG addresses limitations of traditional RAG by combining semantic vector search with intelligent document graph traversal. The system performs two-step retrieval: first using vector search to find relevant documents, then traversing connected documents through extracted references to provide rich augmented context. This approach is essential for comprehensive responses to complex inquiries, particularly in technical domains with interconnected information.

We'll create a multimodal Cortex Search service for document hybrid retrieval, analyze documents to extract cross-references, build a knowledge graph of connected pages, and develop SQL functions for intelligent document traversal. This infrastructure will power a Streamlit application for multimodal multi-hop RAG.

## Building on Multimodal RAG Foundation

**Important Note**: This notebook builds upon the [multimodal RAG pipeline](https://github.com/Snowflake-Labs/cortex-search/blob/main/examples/08_multimodal_rag/cortex_search_multimodal.ipynb) foundation. While multihop RAG can work with various document modalities, multimodal retrieval provides significantly enhanced accuracy for our target queries and content: **operating and maintenance manuals**. These manuals typically contain: 
- **Complex text** with technical terminology and cross-references
- **Tabular data** with specifications, part numbers, and procedures
- **Visual elements** including diagrams, charts, and technical illustrations
- **Mixed content** where text, tables, and images are deeply interconnected

## Overview
The multihop RAG preprocessing involves:
1. **Document Parsing**: Extract text from PDFs and generated multimodal vector embeddings using `PARSE_DOCUMENT` and `AI_EMBED` SQL functions
2. **Vector DB Setup**: Build multimodal and hybrid (semantic and lexical) search with Cortex Search
3. **Document Analysis**: Extract page cross-references using `AI_COMPLETE` SQL function
4. **Graph DB Setup**: Build simple edge table representing page connections, and a recursive SQL function for easy document traversal


### Getting Started

> **📝 Note**: If you've already completed the [multimodal RAG](https://github.com/Snowflake-Labs/cortex-search/blob/main/examples/08_multimodal_rag/cortex_search_multimodal.ipynb) notebook, you can proceed directly using your processed files.

In this notebook, we use a [sample maintenance manual](https://www.graco.com/content/dam/graco/tech_documents/manuals/312/312796/312796EN-S.pdf) that has already been processed through [multimodal RAG](https://github.com/Snowflake-Labs/cortex-search/blob/main/examples/08_multimodal_rag/cortex_search_multimodal.ipynb) notebook. That PDF manual has been split into single-page PDFs and converted to corresponding PNG images.

📁 **Download Files**: [Sample Processed Documents](https://drive.google.com/drive/folders/1OKfqpAts2cXkDZ3ZwosagTJvH3cBcmbu?usp=sharing)

After downloading, upload the files to your Snowflake internal stage maintaining the directory structure:
- `raw_pdf/` - Original PDF document 
- `paged_pdf/` - PDF files for each page e.g. `{document_name}_page_{page_number}.pdf`
- `paged_image/` - PNG images of each page e.g. `{document_name}_page_{page_number}.png`

## Step 1: Setup and Configuration

First, let's set some required parameters to specify which documents needs be processed from your internal stage


In [ ]:
# Configuration - Update these based on your setup
SOURCE_DOCS_STAGE = "@CORTEX_SEARCH_DOCS.DATA.DOCS"
SOURCE_DOCS_PATH = "raw_pdf"
SOURCE_DOCS_PDF_FILTER = "paged_pdf/%.pdf"  # Use to filter paged PDFs for specific document(s)
SOURCE_DOCS_PNG_FILTER = "paged_image/%.png"  # Use to filter paged images for specific document(s)

let's set session context, namely the database and schema, to be used in subsequent SQL code.

In [ ]:
USE DATABASE CORTEX_SEARCH_DOCS;
USE SCHEMA DATA;

In [ ]:
# Import required libraries for analysis steps
import pandas as pd
import json
from snowflake.snowpark.context import get_active_session
session = get_active_session()

print(f"Connected to: {session.get_current_database()}.{session.get_current_schema()}")

# Count documents by querying stage directly
count_query = f"""
SELECT COUNT(*) as document_count
FROM DIRECTORY('{SOURCE_DOCS_STAGE}')
WHERE RELATIVE_PATH LIKE '{SOURCE_DOCS_PDF_FILTER}'
"""

try:
    document_count = session.sql(count_query).collect()[0]['DOCUMENT_COUNT']
    print(f"📄 Found {document_count} documents matching pattern: {SOURCE_DOCS_PDF_FILTER}")
except Exception as e:
    print(f"❌ Error counting documents: {str(e)}")
    print(f"   Stage: {SOURCE_DOCS_STAGE}")
    print(f"   Filter: {SOURCE_DOCS_PDF_FILTER}")

print("✅ Setup complete!")

## Step 2: Document Parsing - Extract Texts and Embed Images

> **📝 Note**: Skip this section if you've already completed [multimodal RAG](https://github.com/Snowflake-Labs/cortex-search/blob/main/examples/08_multimodal_rag/cortex_search_multimodal.ipynb) notebook.

Let's use Snowflake's `PARSE_DOCUMENT` to extract text from all PDFs. We'll later analyze the text to identify and extract cross-references between pages.

In [ ]:
CREATE OR REPLACE TABLE DOCS_PDF_PARSED AS
SELECT
    RELATIVE_PATH AS FILE_NAME,
    REGEXP_SUBSTR(RELATIVE_PATH, '_page_([0-9]+)\.', 1, 1, 'e') as PAGE_NUMBER,
    PARSE_JSON(TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '{{SOURCE_DOCS_STAGE}}',
        RELATIVE_PATH,
        {'mode': 'LAYOUT'}
    ))):content AS PARSE_DOC_OUTPUT
FROM DIRECTORY('{{SOURCE_DOCS_STAGE}}')
WHERE
    RELATIVE_PATH LIKE '{{SOURCE_DOCS_PDF_FILTER}}';

SELECT * FROM DOCS_PDF_PARSED LIMIT 5;

Let's use Snowflake's `AI_EMBED` to create image vector embeddings for all PNGs.

In [ ]:
CREATE OR REPLACE TABLE DOCS_IMAGE_VECTORS AS
SELECT
    RELATIVE_PATH AS FILE_NAME,
    REGEXP_SUBSTR(RELATIVE_PATH, '_page_([0-9]+)\.', 1, 1, 'e') as PAGE_NUMBER,
    AI_EMBED('voyage-multimodal-3', TO_FILE ('{{SOURCE_DOCS_STAGE}}', RELATIVE_PATH)) AS IMAGE_VECTOR,
FROM DIRECTORY('{{SOURCE_DOCS_STAGE}}')
WHERE
    RELATIVE_PATH LIKE '{{SOURCE_DOCS_PNG_FILTER}}';

SELECT * FROM DOCS_IMAGE_VECTORS LIMIT 5;

Let's join the image vectors and parsed texts into a single table, and create a Cortex Search service

In [ ]:
CREATE OR REPLACE TABLE DOCS_JOINED_DATA AS
SELECT
    v.FILE_NAME AS IMAGE_FILEPATH,
    v.PAGE_NUMBER,
    v.IMAGE_VECTOR AS VECTOR_MAIN,
    p.PARSE_DOC_OUTPUT AS TEXT
FROM
    DOCS_IMAGE_VECTORS v
JOIN
    DOCS_PDF_PARSED p
ON
    SPLIT_PART(SPLIT_PART(v.FILE_NAME, '/', -1), '.', 1) = SPLIT_PART(SPLIT_PART(p.FILE_NAME, '/', -1), '.', 1);

Let's create a Cortex Search service with a text index over the textual data (for lexical matching), and a vector index over the vector embeddings (for semantic matching).

In [ ]:
# First, set the warehouse that Cortex Search uses to materialize results when base table changes
CORTEX_SEARCH_WH = 'COMPUTE_WH'

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE DOCS_SEARCH_MULTIMODAL
  TEXT INDEXES TEXT
  VECTOR INDEXES VECTOR_MAIN
  ATTRIBUTES PAGE_NUMBER
  WAREHOUSE='{{CORTEX_SEARCH_WH}}'
  TARGET_LAG='1 day'
AS (
    SELECT 
        TO_VARCHAR(TEXT) AS TEXT, 
        PAGE_NUMBER, 
        VECTOR_MAIN,
        IMAGE_FILEPATH
    FROM DOCS_JOINED_DATA
);

Cortex Search `DOCS_SEARCH_MULTIMODAL` is both hybrid and multimodal:
- Hybrid: keyword (lexical) and vector (semantic) similiarity search
- Multimodal: search across text and images

Congratulations! You have implemented a multimodal retrieval using Cortex Search service.



## Step 3: Document Analysis - Extract Page References

This step uses AI to analyze each document page and extract references to other pages or external links. This is the foundation of our document graph.

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE DOCS_ANALYSIS AS
SELECT 
    IMAGE_FILEPATH,
    PAGE_NUMBER,
    TEXT as page_text,
    AI_COMPLETE(
        model => 'claude-3-7-sonnet',
        prompt => CONCAT(
            'You are a document analysis assistant. For the following text, identify and extract all references to other documents, pages or external links. Examples include: "See page X for...", "See procedure to the left", "See Fig. X", "Refer to table X", "See volume Y".
            
            Return a JSON array where each element is a JSON object with the following structure: 

            {"type": "doc_reference"|"page_reference"|"external_link",
             "value": "the inferred reference (e.g., document name, external URL or page number including current page)", 
             "context": "the full sentence or phrase containing the reference", 
             "explanation": "brief explanation of what the reference points to"}.

            Rules to follow:
            - External link reference value must be a string representing a URL or URI or relative path.
            - Page reference value must be numeric. Exclude strings like "page", "current page" or "next page" from value field.
            - A self-reference is a reference to a table, figure, section, side, or paragraph in the same text. A self-reference should have type set to `page_reference` and value set to the current page number: ', PAGE_NUMBER, '.
            - Do not consider part or spec numbers as page references, unless the text is referencing those as pages or documents.
            - Do not include the page number in the page header or footer as a reference.
            - Do not return duplicate references.

            Text to analyze: ', TEXT
        ),
        response_format => {
            'type':'json',
            'schema':{
                'type' : 'object',
                'properties' : {
                    'references':{
                        'type':'array',
                        'items':{
                            'type':'object',
                            'properties':{
                                'type' : {'type':'string','enum': ['document_reference', 'page_reference', 'external_link']},
                                'value': {'type':'string'},
                                'context': {'type':'string'},
                                'explanation': {'type':'string'}
                            },
                            'required':['type','value' ,'context','explanation']
                        }
                    }
                }
            } 
        },
        model_parameters => {
            'temperature': 0
        }
    ) as llm_response
FROM DOCS_PDF_PARSED
WHERE IMAGE_FILEPATH LIKE '{{SOURCE_DOCS_PDF_FILTER}}'

In [ ]:
# Check the results of the document analysis
analysis_count = session.sql("SELECT COUNT(*) as count FROM DOCS_ANALYSIS").to_pandas()
print(f"✅ Document analysis completed!")
print(f"Analyzed {analysis_count['COUNT'].iloc[0]} document pages")

Let's display the AI-extracted results for some of the pages (8 through 10). We flatten the references array as follows:

In [ ]:
SELECT 
    t.IMAGE_FILEPATH as FILE_NAME,
    t.PAGE_NUMBER,
    reference.value:type::STRING as reference_type,
    reference.value:value::STRING as reference_value,
    reference.value:context::STRING as reference_context,
    reference.value:explanation::STRING as reference_explanation,
    t.llm_response as raw_llm_response
FROM DOCS_ANALYSIS t,
LATERAL FLATTEN(input => t.llm_response:references) reference
WHERE
    t.llm_response IS NOT NULL
    AND t.PAGE_NUMBER IN (8, 9, 10)
ORDER BY 
    t.PAGE_NUMBER;

## Step 4: Build Document Graph - Create Edges Table

Now we'll create the edges table that represents connections between documents. This is the core of our multihop capability.


In [ ]:
CREATE OR REPLACE TABLE DOCS_EDGES (
    SRC_DOC_FILEPATH STRING,       -- Source document path
    SRC_PAGE_NUMBER NUMBER,        -- Source page number
    DST_DOC_FILEPATH STRING,       -- Destination document path (derived from reference)
    DST_PAGE_NUMBER NUMBER,        -- Destination page number (if page reference)
    REFERENCE_TYPE STRING,         -- 'page_reference' or 'external_link'
    REFERENCE_CONTEXT STRING,      -- The context where reference appears
    REFERENCE_EXPLANATION STRING,  -- AI explanation of the reference
    CREATED_AT TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
INSERT INTO DOCS_EDGES (
    SRC_DOC_FILEPATH,
    SRC_PAGE_NUMBER,
    DST_DOC_FILEPATH,
    DST_PAGE_NUMBER,
    REFERENCE_TYPE,
    REFERENCE_CONTEXT,
    REFERENCE_EXPLANATION
)
SELECT 
    t.IMAGE_FILEPATH as SRC_DOC_FILEPATH,
    t.PAGE_NUMBER as SRC_PAGE_NUMBER,
    -- For page references, construct the destination file path
    CASE 
        WHEN reference.value:type::STRING = 'page_reference' 
        THEN REGEXP_REPLACE(t.IMAGE_FILEPATH, '(_page_\\d+)', '_page_' || reference.value:value::STRING)
        ELSE reference.value:value::STRING  -- For external links, store the URL
    END as DST_DOC_FILEPATH,
    -- Extract page number if it's a page reference
    CASE 
        WHEN reference.value:type::STRING = 'page_reference' 
        THEN TRY_CAST(reference.value:value::STRING AS NUMBER)
        ELSE NULL
    END as DST_PAGE_NUMBER,
    reference.value:type::STRING as REFERENCE_TYPE,
    reference.value:context::STRING as REFERENCE_CONTEXT,
    reference.value:explanation::STRING as REFERENCE_EXPLANATION
FROM DOCS_ANALYSIS t,
LATERAL FLATTEN(input => t.llm_response:references) reference
WHERE
    t.llm_response IS NOT NULL
    AND reference.value:type::STRING IS NOT NULL
    AND reference.value:value::STRING IS NOT NULL;


In [ ]:
SELECT REFERENCE_TYPE, COUNT(*) as count 
FROM DOCS_EDGES 
GROUP BY REFERENCE_TYPE

## Step 5: Create the FIND_CONNECTED_PAGES Function

This is the core function that will be used by our Streamlit app for multihop document retrieval.


In [ ]:
CREATE OR REPLACE FUNCTION FIND_CONNECTED_PAGES(
    start_paths ARRAY,
    max_hops INTEGER
)
RETURNS TABLE (
    source_path STRING,
    source_page NUMBER,
    dest_path STRING,
    dest_page NUMBER,
    ref_type STRING,
    ref_count INTEGER,
    explanations ARRAY,
    contexts ARRAY,
    hop_count INTEGER
)
AS
$$
    WITH RECURSIVE connected_images AS (
        -- Base case: start with our initial set of images
        SELECT 
            e.*,
            0 as hop_count
        FROM DOCS_EDGES e
        WHERE ARRAY_CONTAINS(e.SRC_DOC_FILEPATH::VARIANT, START_PATHS)
          AND e.REFERENCE_TYPE = 'page_reference'
          AND e.DST_PAGE_NUMBER IS NOT NULL

        UNION ALL

        -- Recursive case: find images connected to our current set
        SELECT 
            e.*,
            c.hop_count + 1
        FROM DOCS_EDGES e
        JOIN connected_images c 
            ON e.SRC_DOC_FILEPATH = c.DST_DOC_FILEPATH
        WHERE e.REFERENCE_TYPE = 'page_reference'
          AND e.DST_PAGE_NUMBER IS NOT NULL
          AND c.hop_count < MAX_HOPS
    )
    SELECT 
        SRC_DOC_FILEPATH as source_path,
        SRC_PAGE_NUMBER as source_page,
        DST_DOC_FILEPATH as dest_path,
        DST_PAGE_NUMBER as dest_page,
        REFERENCE_TYPE as ref_type,
        COUNT(*) as ref_count,
        ARRAY_AGG(DISTINCT REFERENCE_EXPLANATION) as explanations,
        ARRAY_AGG(DISTINCT REFERENCE_CONTEXT) as contexts,
        MIN(hop_count) as hop_count
    FROM connected_images
    GROUP BY 1, 2, 3, 4, 5
    ORDER BY hop_count, ref_count DESC
$$;

## Step 6: Test the Function & Final Validation

Let's test our function with the same format that will be used by the Streamlit app.


In [ ]:
-- Test the function with sample image paths
SELECT
    DEST_PATH AS dest_path,
    DEST_PAGE AS dest_page,
    ARRAY_AGG(exp.value) AS explanations
FROM TABLE(FIND_CONNECTED_PAGES(
    ARRAY_CONSTRUCT('paged_image/312796EN-S_page_8.png', 'paged_image/312796EN-S_page_13.png'),
    2
)),
LATERAL FLATTEN(input => EXPLANATIONS) exp
GROUP BY 1, 2
ORDER BY DEST_PAGE;

In [ ]:
function_results = step6_test.to_pandas()
print(f"🧪 Function Test Results:")
print(f"Found {len(function_results)} connected documents")

In [ ]:
-- Final verification that all required objects exist

-- Check Cortex Search Service by describing the specific service
DESC CORTEX SEARCH SERVICE DOCS_SEARCH_MULTIMODAL;

SELECT 'DOCS_SEARCH_MULTIMODAL' as object_name,
    'CORTEX SEARCH' as object_type,
    CASE WHEN COUNT(*) > 0 THEN '✅ EXISTS' ELSE '❌ MISSING' END as status
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))

UNION ALL

SELECT 
    'DOCS_ANALYSIS' as object_name,
    'TABLE' as object_type,
    CASE WHEN COUNT(*) > 0 THEN '✅ EXISTS' ELSE '❌ MISSING' END as status
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = CURRENT_SCHEMA() AND TABLE_NAME = 'DOCS_ANALYSIS'

UNION ALL

SELECT 
    'DOCS_EDGES' as object_name,
    'TABLE' as object_type,
    CASE WHEN COUNT(*) > 0 THEN '✅ EXISTS' ELSE '❌ MISSING' END as status
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = CURRENT_SCHEMA() AND TABLE_NAME = 'DOCS_EDGES'

UNION ALL

SELECT 
    'FIND_CONNECTED_PAGES' as object_name,
    'FUNCTION' as object_type,
    CASE WHEN COUNT(*) > 0 THEN '✅ EXISTS' ELSE '❌ MISSING' END as status
FROM INFORMATION_SCHEMA.FUNCTIONS
WHERE FUNCTION_SCHEMA = CURRENT_SCHEMA() AND FUNCTION_NAME = 'FIND_CONNECTED_PAGES'

ORDER BY object_name;


In [ ]:
verification_df = step6_verification.to_pandas()

# Check if all objects exist
all_exist = all(verification_df['STATUS'].str.contains('EXISTS'))
if all_exist:
    print("\n🎉 SUCCESS! All required objects have been created.")
    print("Your multihop RAG preprocessing is complete!")
    print("\n📱 You can now use the Streamlit app: streamlit_chatbot_multihop_rag.py")
else:
    print("\n⚠️ WARNING: Some objects are missing. Please review the errors above.")

## Summary

🎯 **What We've Built:**

1. **Document Parsing Tables** (`DOCS_PDF_PARSED`, `DOCS_IMAGE_VECTORS`): Extracted text content and generated image vectors from document pages using Voyage AI multimodal embedding model
2. **Unified Data Table** (`DOCS_JOINED_DATA`): Combined text and vector data for each document page
3. **Cortex Search Service** (`DOCS_SEARCH_MULTIMODAL`): Hybrid multimodal search service with both text and vector indexes for semantic and lexical retrieval
4. **Document Analysis Table** (`DOCS_ANALYSIS`): AI-extracted references and cross-references from each document page using Claude 3.7 Sonnet 
5. **Document Graph** (`DOCS_EDGES`): Connections between document pages based on extracted references, forming a knowledge graph
6. **Traversal Function** (`FIND_CONNECTED_PAGES`): Recursive SQL function for graph-like page traversal to enable multihop document discovery
7. **Complete Infrastructure**: End-to-end pipeline from raw documents to searchable, connected, and traversable knowledge base

🚀 **Next Steps:**

1. **Deploy the Streamlit App**: Use `streamlit_chatbot_multihop_rag.py` to deploy a multihop RAG application that combines vector search and graph traversal
2. **Test Queries**: Ask questions that require information from multiple connected pages to see multihop retrieval in action
3. **Monitor Performance**: Use the debug mode to see how documents are being retrieved and traversed through the graph
4. **Iterate**: Add more documents, adjust the hop count, or refine the AI reference extraction prompts based on your use case

Your multihop multimodal RAG system is now ready to provide intelligent responses using rich augmented context from both **hybrid vector search** and **document graph traversal**! 🎉
